In [ ]:
import tensorflow as tf
import cv2
import glob
import random
import IPython.display as display
import numpy as np
import pandas as pd
import csv
import time

In [ ]:
train_img_dir = 'data/train_images/'
train_img_suffix = '.jpg'

all_image_paths = list(glob.glob(train_img_dir + '*' + train_img_suffix))
random.shuffle(all_image_paths)

image_count = len(all_image_paths)
print(image_count)

In [ ]:
for n in range(3):
    image_path = random.choice(all_image_paths)
    image_name = image_path.strip(train_img_dir).strip(train_img_suffix)
    print("image id = {}".format(image_name))
    display.display(display.Image(image_path))
    print()

In [ ]:
def get_image(image_path):
    image_id = image_path.strip(train_img_dir).strip(train_img_suffix)
    # import image
    image = tf.io.read_file(image_path)
    # decode
    image = tf.image.decode_image(image)
    # no resizing
    # pick just one channel; normalize
    image = tf.cast(image[:, :, 0], tf.float32)/255.0
    return image_id, image

In [ ]:
img_id, img = get_image(all_image_paths[0])

In [ ]:
img_id

In [ ]:
img

In [ ]:
def make_mask(marker_string, n):
    # initial mask is empty 1D array
    mask = np.zeros(n)
    
    # if markers exist, add detect part according to markers
    if len(marker_string) > 0:
        markers = np.array(marker_string.split(' ')).reshape(-1, 2)
        for marker in markers:
            start = int(marker[0])
            length = int(marker[1])
            mask[start: start+length] = 1.0

    return mask            

In [ ]:
train_mask_file = "data/train.csv"

with open(train_mask_file, "r", newline="") as f:
    reader = csv.reader(f)
    header = next(reader)
    print("CSV header: {}".format(header))
    
    masks = dict()
    for row in reader:
        # read image id and defect type
        img_id, defect_type = row[0].split('.jpg_')
        
        # process mask
        mask_marker_string = row[1]
        mask = make_mask(mask_marker_string, 256*1600)
        
        if img_id in masks:
            masks[img_id][defect_type] = mask
        else:
            masks[img_id] = dict({defect_type: mask})